In [1]:
#instal sdk
%pip install --upgrade --quiet google-cloud-aiplatform

In [4]:
# import library
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [5]:
# get project by CLI
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"
# start vertex
vertexai.init(project=PROJECT_ID, location=REGION)

In [6]:
#Load a generative model gimini pro
model = GenerativeModel("gemini-pro")

In [7]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [8]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```json
{
  "turns": [
    {
      "speaker": "Customer",
      "utterance": "Hi, can I get a cheeseburger and large fries, please?"
    },
    {
      "speaker": "Restaurant employee",
      "utterance": "Coming right up! Anything else you'd like to add to your order?"
    },
    {
      "speaker": "Customer",
      "utterance": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
    },
    {
      "speaker": "Restaurant employee",
      "utterance": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
    }
  ]
}
```


In [9]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "drinks": [
    {
      "item": "orange juice",
      "size": "small",
      "quantity": 1
    }
  ],
  "food": [
    {
      "item": "cheeseburger",
      "quantity": 1
    },
    {
      "item": "fries",
      "size": "large",
      "quantity": 1,
      "modifier": "ketchup on the side"
    }
  ]
}
```


In [10]:
chat = model.start_chat()

In [11]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Monstera Deliciosa Care Guide

The Monstera Deliciosa, also known as the Swiss Cheese Plant, is a popular houseplant known for its large, beautiful leaves with natural holes. It's a relatively low-maintenance plant, but here's a guide to help you keep your Monstera happy and thriving:

**Light:**

* **Bright indirect light:** This is crucial for healthy growth. Place your Monstera near a window that receives plenty of bright, indirect sunlight. Avoid direct sunlight, as it can scorch the leaves.
* **Supplement with artificial light:** If natural light is limited, consider using artificial grow lights to supplement.

**Watering:**

* **Water thoroughly when the top soil feels dry:** This usually means watering every 7-10 days. Don't let the soil dry out completely, but avoid overwatering as well.
* **Check the soil moisture:** The best way to determine when to water is to stick your finger into the top few inches of the soil. If it feels dry, it's time to water.
* **Empty excess wate

In [12]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

Hello! Your monstera deliciosa is here to help you better understand my needs. 

As a houseplant, I need a few key things to thrive. Let’s go over them quickly so you can ensure I live a long and happy life! :

1.  **Light:**  I love bright, indirect light. A spot near a window that gets morning or afternoon sun is perfect. Avoid direct sunlight, as this can scorch my leaves.
2.  **Watering:**  I like my soil to be consistently moist, but not soggy. Water me deeply when the top inch of soil feels dry. Be careful not to overwater, as this can lead to root rot. In winter, I need less water, so allow the soil to dry out more between waterings.
3.  **Humidity:**  I appreciate a humid environment. If the air is dry, you can increase humidity by placing me on a pebble tray filled with water or by using a humidifier.
4.  **Fertilizer:**  I benefit from regular fertilization, especially during the spring and summer growing seasons. Use a balanced houseplant fertilizer and dilute it to half str

In [13]:
#include exemples
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

Likelihood: 3


In [15]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

> Why did the frog get a job at the bank? 

> He was good with his *mon-eys*. 



In [16]:
# fallback response
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!


In [17]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

Ceramic and porcelain are both types of pottery, but they have some key differences. Ceramic is a more general term that refers to any object made from clay that has been hardened by heat. Porcelain is a type of ceramic that is made from a specific type of clay called kaolin, which gives it a white, translucent appearance. Porcelain is also harder and more durable than other types of ceramic.


In [19]:
#contextual information
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

## Aisle Numbers:

Here's where you can typically find the items you listed:

* **Paper plates:** Aisle 4 (Paper and Plastic Products)
* **Mustard:** Aisle 3 (Condiments)
* **Potatoes:** Aisle 9 (Produce) 

**Please note:** This is just a general guide. The aisle numbers may vary depending on the specific grocery store you visit. It's always best to check the store directory or ask an employee for assistance if you can't find something. 



In [20]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

## Michael's Grocery Store Aisle Numbers:

Based on your provided context:

* **Paper plates:** You can find paper plates in **Aisle 17 (Household & Cleaning Supplies)**. 
* **Mustard:** You can find mustard in **Aisle 8 (Condiments & Spices)**.
* **Potatoes:** You can find potatoes in two locations:
    * **Aisle 2 (Vegetables)** if you're looking for fresh potatoes.
    * **Aisle 14 (Frozen Foods)** if you're looking for frozen potatoes. 



In [21]:
## structure prompts with prefixes or tags

prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

Allison, I think I may have found your perfect match! His name is Felix. Not only does he share your love of classical music, especially Beethoven, but he's also an excellent cook who specializes in German cuisine, just like you! Plus, he used to be a water polo player, so he's athletic and loves being by the water, just like you enjoy swimming. It sounds like you two could have amazing conversations about music, share delicious meals together, and even enjoy fun beach outings. You should definitely check him out! 



In [22]:
# system instructions
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-1.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

Oh my, what a question for a music enthusiast like myself!  There's a whole world of incredible musicians worth studying, each offering unique insights into the power and beauty of music. 

Where should we start?  

* **Classical fans** could dive into the complex polyphony of **Johann Sebastian Bach**, the dramatic symphonies of **Ludwig van Beethoven**, or the groundbreaking operas of **Wolfgang Amadeus Mozart**. 

* Want to explore the evolution of **jazz**?  Listen closely to the improvisational genius of **Louis Armstrong**, the bebop mastery of **Charlie Parker**, or the smooth sounds of **Miles Davis**.

*  Interested in the roots of **rock and roll**?  You'll want to get acquainted with pioneers like **Chuck Berry**, **Little Richard**, and **Sister Rosetta Tharpe**.  

*  And don't forget the incredible women in music!  From the soulful vocals of **Aretha Franklin** to the songwriting prowess of **Carole King** and the boundary-breaking performances of **Beyoncé**, their contr

In [23]:
#Chain-of-Thought
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day (Reconfigured factory)
* Low efficiency factories: 1 factory * 30 units/day/factory * 0.5 = 15 units/day (Outaged factory)
* **Total production today: 300 units/day + 60 units/day + 15 units/day = 375 units/day**



In [24]:
#Break down complex tasks
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

## TPUs vs. GPUs: 5 Metaphors

1. **TPUs are like data center sprinters, while GPUs are marathon runners.** TPUs excel at specific tasks with high processing power, making them ideal for training deep learning models. GPUs, with their sustained performance, are better suited for tasks requiring long-term computation, like scientific simulations.

2. **TPUs are specialized racing cars, while GPUs are powerful SUVs.** TPUs are custom-built for machine learning, offering unparalleled speed and efficiency for specific tasks. GPUs are versatile tools capable of handling various computing needs, but with less specialization and efficiency compared to TPUs.

3. **TPUs are like laser scalpels, while GPUs are like Swiss army knives.** TPUs are highly focused on deep learning, making them precise and efficient for specific tasks. GPUs are versatile with broader capabilities but less accuracy and efficiency than TPUs in their domain.

4. **TPUs are like a team of specialized doctors, while GPUs a

In [25]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

## The Pit Crew vs. Mechanic Metaphor: Visualizing the Difference Between TPUs and GPUs

From a college student's perspective, the **Formula 1 pit crew versus local garage mechanic** metaphor offers a clear and compelling visual to understand the key differences between TPUs and GPUs. Imagine this:

**TPUs:**  A Formula 1 pit crew, expertly trained and equipped with specialized tools. Each member has a specific role, working in perfect synchronicity to achieve a singular goal: a lightning-fast pit stop. They are highly efficient, completing complex tasks with astonishing speed and precision. 

**GPUs:**  A skilled mechanic at a local garage, capable of handling various repairs and maintenance tasks. They have a broad range of tools and knowledge, making them adaptable to diverse situations. However, due to the wider scope of their expertise, they may not be as fast or specialized as the pit crew for a specific task.

This analogy effectively captures the essence of both technologies:



In [26]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

## The Pit Stop to Machine Learning Acceleration: Understanding TPUs and GPUs

Ever watched a Formula 1 pit stop? It's a breathtaking display of teamwork and precision. In mere seconds, a team of experts equipped with specialized tools refuels the car, changes tires, and sends it back on the track – all in a perfectly synchronized ballet of motion.

Now, imagine that pit crew as a **Tensor Processing Unit (TPU)** – a custom-designed machine learning accelerator built for speed and efficiency. Just like the pit crew focuses on a singular goal, the TPU excels at specific deep learning tasks, utilizing multiple cores to achieve lightning-fast computation. 

But what about your local garage mechanic? They may not have the high-tech tools of a Formula 1 pit crew, but they possess a diverse skillset and can handle a wide range of repairs. This versatility is akin to the **Graphics Processing Unit (GPU)** – a powerful tool capable of handling various computing tasks. While GPUs may not be as 